## 201810824 현소미 12주차 과제

## 컨트랙에 이벤트를 추가해서 구현

In [7]:
%%writefile src/Multiply7Event.sol
pragma solidity ^0.5.10;
contract Multiply7Event {
    
    event Print(address from, uint timestamp, uint result);

    function multiply(uint input) public {
        7*input;
        emit Print(msg.sender, now, 7*input);
    }
}

Overwriting src/Multiply7Event.sol


In [8]:
!solc --gas --abi --bin src/Multiply7Event.sol


======= src/Multiply7Event.sol:Multiply7Event =======
Gas estimation:
construction:
   105 + 52400 = 52505
external:
   multiply(uint256):	1849
Binary: 
6080604052348015600f57600080fd5b506101068061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c6888fa114602d575b600080fd5b605660048036036020811015604157600080fd5b81019080803590602001909291905050506058565b005b7f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334283600702604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a15056fea265627a7a72305820f3f67738d620fa9a8a030e4ea4a01cd4967fb81b877c9af7c1b8c95a471386cc64736f6c634300050a0032
Contract JSON ABI 
[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{

In [9]:
%%writefile src/Multiply7EventDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var _abiArray = JSON.parse(_abiStr);
var _abiArray =[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"timestamp","type":"uint256"},{"indexed":false,"name":"result","type":"uint256"}],"name":"Print","type":"event"}];
var _bin = "0x" + "6080604052348015600f57600080fd5b506101068061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c6888fa114602d575b600080fd5b605660048036036020811015604157600080fd5b81019080803590602001909291905050506058565b005b7f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334283600702604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a15056fea265627a7a72305820f3f67738d620fa9a8a030e4ea4a01cd4967fb81b877c9af7c1b8c95a471386cc64736f6c634300050a0032";
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/Multiply7EventDeploy.js


In [10]:
!node src/Multiply7EventDeploy.js

Deploying the contract from 0xc85Ea5B8516560Ba5Fc5A9FA10407d70bf8C1B03
hash: 0xa83594cc8a7216abfecee723ee4ff2bcbc279064b2e22aab3858ae6857df8743
---> The contract deployed to: 0x4eF3925C27cA082f04Ec3B6d9891D4470d4031AB


In [11]:
%%writefile src/Multiply7EventUse.js
var Web3=require('web3');
var web3 ;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
}
var _abiArray=[{"constant":false,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"timestamp","type":"uint256"},{"indexed":false,"name":"result","type":"uint256"}],"name":"Print","type":"event"}];
var Multiply7Event = new web3.eth.Contract(_abiArray, "0x4eF3925C27cA082f04Ec3B6d9891D4470d4031AB");
var event = Multiply7Event.events.Print({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) );
    }
});


async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const value = await Multiply7Event.methods.multiply(8).send({from: accounts[0], gas: 3000000 , gasPrice: '1000000000'});
    console.log("---> myFunction called "+JSON.stringify(value)+
        '\n---> '+ JSON.stringify(value.events.Print.returnValues));
     const value2 = await Multiply7Event.methods.multiply(16).send({from: accounts[0], gas: 3000000 , gasPrice: '1000000000'});
    console.log("---> myFunction called "+JSON.stringify(value2)+
        '\n---> '+ JSON.stringify(value2.events.Print.returnValues));

    
}
doIt()

Overwriting src/Multiply7EventUse.js


## 배포하고 사용한 후 곱셈 결과값과 주소, 시간 출력함

In [12]:
!node src/Multiply7EventUse.js

Account: 0xc85Ea5B8516560Ba5Fc5A9FA10407d70bf8C1B03
Event fired: {"logIndex":0,"transactionIndex":0,"transactionHash":"0x25a66f91db79a30d90ae97b12e68b2dc3322eb9d38475721947a8b13d3760bf9","blockHash":"0x49d33a44c27e0f987d7674a42320cf3fd5038a4c5f059223c8b0873c53b3a3cc","blockNumber":2,"address":"0x4eF3925C27cA082f04Ec3B6d9891D4470d4031AB","type":"mined","id":"log_3641241c","returnValues":{"0":"0xc85Ea5B8516560Ba5Fc5A9FA10407d70bf8C1B03","1":"1591408650","2":"56","from":"0xc85Ea5B8516560Ba5Fc5A9FA10407d70bf8C1B03","timestamp":"1591408650","result":"56"},"event":"Print","signature":"0x91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205","raw":{"data":"0x000000000000000000000000c85ea5b8516560ba5fc5a9fa10407d70bf8c1b03000000000000000000000000000000000000000000000000000000005edaf80a0000000000000000000000000000000000000000000000000000000000000038","topics":["0x91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205"]}}
---> myFunction called {"transactionHash":"0x25a66f91